In [95]:
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer

FILE = 'AOD'
df = pd.DataFrame.from_csv('./Data/'+FILE+'.csv')
df.content = df.content.str.replace('/',' ') 
df.content = df.content.str.replace(',',' ') 
df.content = df.content.str.replace('(',' ')
df.content = df.content.str.replace(')',' ') 

C:\Users\Prime\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """


In [96]:
len(df)

547

In [97]:
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer 
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

stop_words = set(stopwords.words('english'))

from tqdm._tqdm_notebook import tqdm_notebook
import pandas as pd
tqdm_notebook.pandas()

In [98]:
# Convert to list
FILE = FILE.replace('_',' ')
data = df[(df.count_catogory_no_duplicate==1) & (df[FILE]==True)].content.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

# Remove single word number quotes
#data = [re.sub("\d+.\d*", "", sent) for sent in data]

# Remove distracting single quotes
data = [re.sub(":", " ", sent) for sent in data]
data = [re.sub('/', " ", sent) for sent in data]

# Remove punctation
data = [re.sub(r'[^\w\s]','',sent) for sent in data]
#data = [re.sub("\!", "", sent) for sent in data]

# Remove blank 
temp = [x for x in data if x != '']

data = temp

In [99]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [100]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [101]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [102]:
flat_list = [item for sublist in data_words_bigrams for item in sublist]

In [103]:
word_dict = {}
for word in flat_list: 
    word_dict[word] = word_dict.get(word, 0) + 1

In [104]:
word_freq = []
for key, value in word_dict.items():
    word_freq.append((value, key))

In [105]:
word_freq.sort(reverse=True) 
print(word_freq[:4])

[(3487, 'dissolution'), (2824, 'corporation'), (1772, 'number'), (1309, 'approval')]


In [106]:
def filtered_content(row):
    temp = {}
    for w in row.split():
        if w in word_dict:
            if word_dict[w]>10:
                temp[w] = word_dict[w]

    sorted_list = []
    for key, value in sorted(temp.items(), key=lambda kv:kv[1], reverse=True):
        sorted_list.append(key)

    return sorted_list

In [107]:
df['content_filtered'] = df.content.progress_apply(filtered_content)

In [108]:
df = df.sort_values(by='content_filtered') 

In [109]:
df[['state', 'id', 'link', 'county', 'newspaper', 'date', 'content', 'notice_id', 'category', 'content_filtered']].to_csv('./Data/'+FILE+'_Filtered.csv')